In [45]:
%matplotlib inline
from PIL import Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from collections import defaultdict

from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import fbeta_score, precision_score, make_scorer, average_precision_score

import cv2

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras import backend as K

In [46]:
n_samples = 1000
rescaled_dim = 32

path = '/Users/mereltheisen'
df = pd.read_csv(path + '/Kaggle/train.csv')
sample = pd.read_csv(path + '/Dropbox/Kaggle/sample_submission.csv')
lb = MultiLabelBinarizer()


In [47]:
def set_output(df):
    df['split_tags'] = df['tags'].map(lambda row: row.split(" "))
    y = lb.fit_transform(df['split_tags'])
    y = y[:n_samples]
    return y

def set_input(train_features):
    X = np.squeeze(np.array(train_features, np.float16) / 255.)
    return X

def calculate_f2(y_test, predicted):
    score = fbeta_score(y_test, predicted, beta=2, average=None)
    avg_sample_score = fbeta_score(y_test, predicted, beta=2, average='samples')
    print('Average F2 test score {}'.format(avg_sample_score))
    #print('F2 test scores per tag:')
    #print [(lb.classes_[l], score[l]) for l in score.argsort()[::-1]]

In [48]:
# FEATURE EXTRACTION

# Raw features
#raw_features_train = [cv2.resize(cv2.imread(path + '/Kaggle/train-jpg/{}.jpg'.format(name)), (rescaled_dim, rescaled_dim)) for name in df.head(n_samples)['image_name'].values]
raw_features_train = [cv2.imread(path + '/Kaggle/train-jpg/{}.jpg'.format(name)) for name in df.head(n_samples)['image_name'].values]

#raw_features_test = [cv2.resize(plt.imread(path + '/Kaggle/test-jpg/{}.jpg'.format(name)), (rescaled_dim, rescaled_dim)).reshape(1, -1) for name in sample['image_name'].values]

In [49]:
# SET X's & Y's
y = set_output(df)
X = set_input(raw_features_train)
#X_sub = set_input(raw_features_test)

print(X.shape, y.shape, lb.classes_)
#print X_sub.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train.shape

((1000, 256, 256, 3), (1000, 17), array(['agriculture', 'artisinal_mine', 'bare_ground', 'blooming',
       'blow_down', 'clear', 'cloudy', 'conventional_mine', 'cultivation',
       'habitation', 'haze', 'partly_cloudy', 'primary', 'road',
       'selective_logging', 'slash_burn', 'water'], dtype=object))


(670, 256, 256, 3)

In [51]:
# create the base pre-trained model
base_model = InceptionV3(input_shape=(256,256,3), weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 17 classes
predictions = Dense(17, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
#print model.summary()

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit(X_train, y_train, batch_size=100, epochs=2)

print model.metrics_names
model.evaluate(X_test, y_test)

predicted = model.predict(X_test, batch_size=100, verbose=0)

Epoch 1/2
670/670 [==============================] - 211s - loss: 0.6820 - acc: 0.7992    
Epoch 2/2
670/670 [==============================] - 193s - loss: 0.3226 - acc: 0.9048    
['loss', 'acc']
330/330 [==============================] - 101s   


In [52]:
predicted = (predicted > 0.5).astype(int)
calculate_f2(y_test, predicted)

Average F2 test score 0.669870588365


In [23]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(X_train, y_train, batch_size=100, epochs=2)

(0, 'input_6')
(1, 'conv2d_217')
(2, 'batch_normalization_215')
(3, 'activation_215')
(4, 'conv2d_218')
(5, 'batch_normalization_216')
(6, 'activation_216')
(7, 'conv2d_219')
(8, 'batch_normalization_217')
(9, 'activation_217')
(10, 'max_pooling2d_11')
(11, 'conv2d_220')
(12, 'batch_normalization_218')
(13, 'activation_218')
(14, 'conv2d_221')
(15, 'batch_normalization_219')
(16, 'activation_219')
(17, 'max_pooling2d_12')
(18, 'conv2d_225')
(19, 'batch_normalization_223')
(20, 'activation_223')
(21, 'conv2d_223')
(22, 'conv2d_226')
(23, 'batch_normalization_221')
(24, 'batch_normalization_224')
(25, 'activation_221')
(26, 'activation_224')
(27, 'average_pooling2d_22')
(28, 'conv2d_222')
(29, 'conv2d_224')
(30, 'conv2d_227')
(31, 'conv2d_228')
(32, 'batch_normalization_220')
(33, 'batch_normalization_222')
(34, 'batch_normalization_225')
(35, 'batch_normalization_226')
(36, 'activation_220')
(37, 'activation_222')
(38, 'activation_225')
(39, 'activation_226')
(40, 'mixed0')
(41, 'conv2d